In [1]:
import re
# from nltk.corpus import stopwords
from scipy import sparse
import numpy as np
# import holidays
from utils import ExtractCalendar
from datetime import date
import re
import pandas as pd
from datetime import date
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import torch
import torch.nn as nn

In [6]:
invoice_data = pd.read_csv('./data/shwapno/invoice_data.csv')
invoice_data.ProductCode = invoice_data.ProductCode.astype(str)

baskets = invoice_data.loc[invoice_data.index.repeat(invoice_data['SalesQTY'])].groupby('Invoiceno')['ProductCode'].apply(list)
products_in_basket = invoice_data.loc[invoice_data.index.repeat(invoice_data['SalesQTY'])].groupby('Invoiceno')['ProductName'].apply(list)

other_elements = invoice_data[['Invoiceno','PrepareDate', 'WeekEndFlag', 'DayTimeFlag']].drop_duplicates().set_index(['Invoiceno'])
other_elements['PrepareDate'] = pd.to_datetime(other_elements.PrepareDate)

basket_corpus = pd.concat([baskets, products_in_basket, other_elements], axis=1)
daytime_dict = {key: i for i, key in enumerate(basket_corpus.DayTimeFlag.unique())}
basket_corpus.DayTimeFlag = basket_corpus.DayTimeFlag.map(daytime_dict)
basket_corpus

,ProductCode,ProductName,PrepareDate,WeekEndFlag,DayTimeFlag
Invoiceno,,,,,
D0071501010177,"[2500464, 2805315, 2805913, 2805913, 3100861]","[AARONG UHT Full Cream Liquid Milk 500ml, Prot...",2015-01-01 09:44:08.857,0,0
D0071501010182,"[2700164, 2700327, 2702241, 2500234]","[Radhuni Panch Foran 50 gm, Pran Thai Chilli S...",2015-01-01 09:56:30.700,0,0
D0071501010192,"[3901391, 4203132, 4203156, 2802216, 2802216, ...","[S201011 Party Spray Cragy Ribbon-PS1/2, S2010...",2015-01-01 10:08:11.857,0,0
D0071501010248,"[2805453, 2700122, 2400450, 2805452, 2700154, ...","[IFAD Eggy Instant Noodles Chicken 65Gm, Radhu...",2015-01-01 11:18:25.920,0,0
D0071501010311,"[2801349, 2801347, 2802229, 2300267]","[Golden Harvest Dal Puri 454gm (10 Pcs), GH De...",2015-01-01 12:07:57.460,0,0
...,...,...,...,...,...
D0072209191015,"[2816455, 2816455, 2815330, 2815330, 2815277, ...","[Fresh Healthy Choice Salted Biscuit 100g, Fre...",2022-09-19 21:14:46.067,0,3
D0072209191029,"[2603361, 2603361, 2603361, 2900006, 2900006, ...","[Wheel Laundry Soap 125g, Wheel Laundry Soap 1...",2022-09-19 21:39:07.430,0,3
D0072209191033,"[3017647, 3000680, 3811722, 3000382]","[Vaseline Mosquito Defence Original 50ml, Clos...",2022-09-19 21:42:29.560,0,3


In [10]:
calendar = ExtractCalendar([2015, 2016, 2017, 2018, 2019, 2021, 2022], 'bangladesh')
def is_festival(_date):
    return _date in calendar.festivals

basket_corpus = pd.concat([basket_corpus, basket_corpus.PrepareDate.rename('FestivalFlag').apply(is_festival).astype(int)], axis=1)

C:\Users\shadm\AppData\Local\Temp\ipykernel_25484\3142944248.py:3: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  return _date in calendar.festivals


In [9]:
basket_corpus

,ProductCode,ProductName,PrepareDate,WeekEndFlag,DayTimeFlag
Invoiceno,,,,,
D0071501010177,"[2500464, 2805315, 2805913, 2805913, 3100861]","[AARONG UHT Full Cream Liquid Milk 500ml, Prot...",2015-01-01 09:44:08.857,0,0
D0071501010182,"[2700164, 2700327, 2702241, 2500234]","[Radhuni Panch Foran 50 gm, Pran Thai Chilli S...",2015-01-01 09:56:30.700,0,0
D0071501010192,"[3901391, 4203132, 4203156, 2802216, 2802216, ...","[S201011 Party Spray Cragy Ribbon-PS1/2, S2010...",2015-01-01 10:08:11.857,0,0
D0071501010248,"[2805453, 2700122, 2400450, 2805452, 2700154, ...","[IFAD Eggy Instant Noodles Chicken 65Gm, Radhu...",2015-01-01 11:18:25.920,0,0
D0071501010311,"[2801349, 2801347, 2802229, 2300267]","[Golden Harvest Dal Puri 454gm (10 Pcs), GH De...",2015-01-01 12:07:57.460,0,0
...,...,...,...,...,...
D0072209191015,"[2816455, 2816455, 2815330, 2815330, 2815277, ...","[Fresh Healthy Choice Salted Biscuit 100g, Fre...",2022-09-19 21:14:46.067,0,3
D0072209191029,"[2603361, 2603361, 2603361, 2900006, 2900006, ...","[Wheel Laundry Soap 125g, Wheel Laundry Soap 1...",2022-09-19 21:39:07.430,0,3
D0072209191033,"[3017647, 3000680, 3811722, 3000382]","[Vaseline Mosquito Defence Original 50ml, Clos...",2022-09-19 21:42:29.560,0,3


In [ ]:
basket_corpus[['ProductCode', 'ProductName', 'WeekEndFlag', 'DayTimeFlag', 'FestivalFlag']]

In [5]:
invoice_data[['SalesQTY']] = invoice_data[['SalesQTY']].astype(int)
product_code2name =invoice_data[['ProductCode','ProductName']].drop_duplicates().set_index('ProductCode').to_dict()['ProductName']
product_name2code =invoice_data[['ProductCode','ProductName']].drop_duplicates().set_index('ProductName').to_dict()['ProductCode']
unique_product_code = [value for key, value in invoice_data[['ProductCode']].drop_duplicates(ignore_index=True).to_dict()['ProductCode'].items()]
unique_product_name = [value for key, value in invoice_data[['ProductName']].drop_duplicates(ignore_index=True).to_dict()['ProductName'].items()]

In [6]:
product_code2name
product_name2code
unique_product_name

['AARONG UHT Full Cream Liquid Milk 500ml',
 'Protiva Chui Pitha 200g',
 'Satej Coconut Naru 200g',
 'X-Shonalika Chicken Dressed Large (Pcs)',
 'Radhuni Panch Foran 50 gm',
 'Pran Thai Chilli Sauce 340gm',
 'Pran Soy Sauce 650ml',
 'No.1Sweetened condensed milk 400 gm',
 'S201011 Party Spray Cragy Ribbon-PS1/2',
 'S201011 Love Balloon Thai 20 Pcs BL 1/7',
 'S201011 Balloon Thai BL 1/2',
 'MARS Fun Size 18gm',
 'IFAD Eggy Instant Noodles Chicken 65Gm',
 'Radhuni Turmeric Regular 50 gm',
 'Shwapno Sugar (P) 1Kg',
 'IFAD Eggy Instant Noodles Masala 65Gm',
 'Radhuni Chilli 100g',
 'ACI Pure Salt 1kg',
 'Harpic Power Plus 500 ML',
 'Ispahani Mirzapore Bestleaf 200g(P.Bag)',
 'Mama Instant Noodles Chicken Flav. 248g',
 'Rupchanda Soyabean Oil 2ltr',
 'Teer Suji 500gm',
 'Golden Harvest Dal Puri 454gm (10 Pcs)',
 'GH Deshi Paratha 20pcs 1300g',
 'Ruchi Barbecue Chanachur 170 gm',
 'Mac Coffee Mix 3B1 Original 20g 25pcs',
 'Shan Karachi Beef Biryani Mix 60g',
 'Cerelac3 Fruits, Wheat & Milk 4

## Initiate Torch pipeline

In [60]:
window = 3
baskets_tensor = [torch.combinations(torch.tensor(basket), r=window ).numpy() for basket in basket_corpus['ProductCode'].tolist()]

baskets_tensor
# torch.tensor(basket_corpus['ProductCode'].tolist()[0])
# torch.tensor(['kjsk'])
# x = torch.tensor(basket_corpus['ProductName'].tolist())
# torch.permute(x, (2,2))

# x = torch.randn(5)
# f = torch.combinations(x, 2)
# x.size()
# torch.Size([2, 3, 5])
# torch.permute(x, (3, 2)).size()
# for text in texts:
#     text = clean_text(text)
#     all_text += text
#     for i, word in enumerate(text):
#         for w in range(window):
#             if i + 1 + w < len(text):
#                 word_lists.append([word] + [text[(i + 1 + w)]])
#             if i - w - 1 >= 0:
#                 word_lists.append([word] + [text[(i - w - 1)]])

# f

[array([[2500464, 2805315, 2805913],
        [2500464, 2805315, 2805913],
        [2500464, 2805315, 3100861],
        [2500464, 2805913, 2805913],
        [2500464, 2805913, 3100861],
        [2500464, 2805913, 3100861],
        [2805315, 2805913, 2805913],
        [2805315, 2805913, 3100861],
        [2805315, 2805913, 3100861],
        [2805913, 2805913, 3100861]]),
 array([[2700164, 2700327, 2702241],
        [2700164, 2700327, 2500234],
        [2700164, 2702241, 2500234],
        [2700327, 2702241, 2500234]]),
 array([[3901391, 4203132, 4203156],
        [3901391, 4203132, 2802216],
        [3901391, 4203132, 2802216],
        [3901391, 4203132, 2802216],
        [3901391, 4203132, 2802216],
        [3901391, 4203132, 2802216],
        [3901391, 4203132, 2802216],
        [3901391, 4203132, 2802216],
        [3901391, 4203156, 2802216],
        [3901391, 4203156, 2802216],
        [3901391, 4203156, 2802216],
        [3901391, 4203156, 2802216],
        [3901391, 4203156, 2802216

In [ ]:
basket_numpy = [torch.tensor.numpy(basket) for basket in baskets_tensor]

In [44]:
x

tensor([-1.1602, -0.7048,  0.2778, -0.7635,  0.9516])

In [18]:


n = len(unique_product_code)
d = 5

# embedding = nn.Embedding(num_embeddings=n,
#                          embedding_dim=d,
#                          scale_grad_by_freq=True)
# W =
#
# basket_corpus['ProductName'].tolist()

In [19]:
W = torch.randn((10, d), requires_grad=True)

In [20]:
W

tensor([[-1.7055, -0.9663,  0.9282,  2.0120,  0.8026],
        [ 0.5733,  0.3174,  0.1740,  0.1068, -1.1529],
        [-0.6616, -0.5059,  0.8819,  0.1402,  0.7931],
        [ 1.2246,  0.1315,  0.9517,  0.7330, -0.0452],
        [-1.1456,  0.2103,  0.4920, -0.9144,  0.1835],
        [ 1.6325, -1.2538, -1.2988, -0.2099,  1.3304],
        [-1.0015, -0.0525, -1.2435, -1.0118, -0.5732],
        [ 1.1298, -0.0454, -0.7331,  0.8635,  0.1417],
        [ 0.2679,  0.2550, -1.8900, -1.0946,  2.8826],
        [-0.2809, -0.0591, -0.7665, -1.1357,  1.3744]], requires_grad=True)

In [22]:
idx = torch.tensor([1, 2])
idx

tensor([1, 2])